In [ ]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Aidan\out\temp")
    os.remove(r"E:\TMP\Aidan\out\y00006400.x00019200.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(2)

# run fnt_cube_processor.py
! python .\fnt_cube_processor.py -i E:\TMP\Aidan\tmp -o E:\TMP\Aidan\out -n 1 -d  
# time.sleep(2)
img, header = read(r"E:\TMP\Aidan\out\y00006400.x00019200.nrrd")
imwrite(r"E:\TMP\Aidan\out\test_decon.tif", img, compression=1)

In [1]:
def convert_file(input_nrrd):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

In [1]:
# Testing deconvolutions for real folder (this one takes a long time)
from nrrd import read
from tifffile import imwrite
from pathlib import Path
from os import system 
from multiprocessing import Pool


# running fnt_cube_processor with 8 threads
input_folder = r"E:\TMP\Aidan\z00003200"
output_nrrd_folder = r"E:\TMP\Aidan\z00003200_deconv_nrrd"
output_tif_folder = r"E:\TMP\Aidan\z00003200_tif"
num_threads = 8

system(fr"python .\fnt_cube_processor.py -i {input_folder} -o {output_nrrd_folder} -n {num_threads} -d")

# convert to tif files
nrrd_paths = [f for f in Path(output_nrrd_folder).iterdir() if f.is_file()]

# convert single file to tif
def convert_file(input_nrrd: Path):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

for path in nrrd_paths:
    convert_file(path)

print("All nrrd -> tif conversions completed successfully")


All nrrd -> tif conversions completed successfully
